In [ ]:
# Transformer 요약 모델 & 파인 튜닝
# - Hugging Face 라이브러리 적용
# - AI HUB 요약문 및 레포트 뉴스(news) 데이터셋 적용
# - 입력된 문장을 요약 모델을 통한 문장 요약
# 1. 학습 목표
# - 구조 최적화 및 파이프라인 단순화
# - AI HUB 요약문 및 레포트 뉴스(news) 데이터셋 전처리
# - 병렬 문장쌍 데이터셋 변환 전처리
# - 토크나이징 및 토크나이징 전처리
# - 베이스 모델 로드
# - LoRA(Low-Rank Adaptation) 설정, 특정 레이어에 작은 저차원 행렬(랭크 r)을 삽입해서 학습
# - LoRA(Low-Rank Adaptation) 모델, 메모리 효율성/빠른 학습/도메인 적용, base 모델에 여러 LoRA 모듈을 붙였다 떼었다 할 수 있음
# - 학습 args 설정
# - Trainer 정의
# - Trainer 실행
# - LoRA 적용된 모델 저장, LoRA모델/토크나이저
# - LoRA 적용된 모델 불러오기, 베이스모델/LoRA모델/토크나이저

In [1]:
import torch
import numpy as np
import glob, json, re, os, random, csv

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch.__version__, device)

print("CUDA 사용 가능 여부:", torch.cuda.is_available())
print("PyTorch CUDA 버전:", torch.version.cuda)
print("빌드 정보:", torch.__version__)
if torch.cuda.is_available():
    print("사용 중인 GPU:", torch.cuda.get_device_name(0))

2.6.0 cpu
CUDA 사용 가능 여부: False
PyTorch CUDA 버전: None
빌드 정보: 2.6.0


In [ ]:
# 데이터셋 전처리 - AI HUB 요약문 및 레포트 뉴스(news) 데이터셋 적용
import os
import json
import pandas as pd

# 기본 경로
base_dir = './llm_data/ai_hub_summary_news_r'

def parse_json_folder(input_dir, summary_type):
    rows = []
    for file_name in os.listdir(input_dir):
        if not file_name.endswith('.json'):
            continue
        file_path = os.path.join(input_dir, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        passage = data['Meta(Refine)']['passage']
        doc_type = data['Meta(Acqusition)']['doc_type']

        if summary_type == 'short':
            for key in ['summary1', 'summary2']:
                summary = data['Annotation'].get(key)
                if summary:
                    rows.append({
                        'src': passage,
                        'tgt': summary,
                        'summary_type': 'short',
                        'domain': doc_type
                    })
        elif summary_type == 'long':
            summary3 = data['Annotation'].get('summary3')
            if summary3:
                rows.append({
                    'src': passage,
                    'tgt': summary3,
                    'summary_type': 'long',
                    'domain': doc_type
                })
    # 판다스 데이터프레이으로 생성 후 리턴
    return pd.DataFrame(rows)

# 학습 데이터 로딩
train_short = parse_json_folder(os.path.join(base_dir, 'train/2-3sent'), 'short')
train_long = parse_json_folder(os.path.join(base_dir, 'train/20per'), 'long')

# 검증 데이터 로딩
valid_short = parse_json_folder(os.path.join(base_dir, 'valid/2-3sent'), 'short')
valid_long = parse_json_folder(os.path.join(base_dir, 'valid/20per'), 'long')

# 확인
print('train short:', train_short)
print('train long:', train_long)
print('valid short:', valid_short)
print('valid long:', valid_long)

train short:                                                      src  \
0      오거돈 전 부산시장의 성추행 사건이 불거진 후 정치권을 중심으로 여권이 이 사실을 ...   
1      오거돈 전 부산시장의 성추행 사건이 불거진 후 정치권을 중심으로 여권이 이 사실을 ...   
2       “동선 봐도 도움이 안 되는 것 같아요.\n  어딘지 알아야 피해가고 조심할 텐데...   
3       “동선 봐도 도움이 안 되는 것 같아요.\n  어딘지 알아야 피해가고 조심할 텐데...   
4      대구 대실요양병원에서 21일 신종 코로나바이러스 감염증(코로나19) 확진자가 52명...   
...                                                  ...   
21595   병원들은 지난 21일부터 전공의들이 단계적으로 파업에 돌입한 데 따라 인력 부족으...   
21596     1일 0시 기준 수도권 신규 환자는 서울(45명)·경기(36명)·인천(0명)으...   
21597     1일 0시 기준 수도권 신규 환자는 서울(45명)·경기(36명)·인천(0명)으...   
21598  권준욱 질병관리청 중앙방역대책본부(방대본) 부본부장이 "젊은 남성 운동선수들도 코로...   
21599  권준욱 질병관리청 중앙방역대책본부(방대본) 부본부장이 "젊은 남성 운동선수들도 코로...   

                                                     tgt summary_type  domain  
0      오 전 부산시장의 성추행 사건이 불거진 후 여권이 이 사실을 사전에 몰랐을까 하는 ...        short  news_r  
1      오거돈 전 부산시장의 성추행 사건이 불거진 후 정치권을 중심으로 여권이 이 사실을 ...        short  news_r  
2      코로나19 확진자 동선은 사생활 침

In [5]:
# 학습 데이터 저장
train_short.to_csv(os.path.join(base_dir, 'train/2-3sent/train.csv'), index=False, encoding='utf-8-sig')
train_long.to_csv(os.path.join(base_dir, 'train/20per/train.csv'), index=False, encoding='utf-8-sig')

# 검증 데이터 저장
valid_short.to_csv(os.path.join(base_dir, 'valid/2-3sent/valid.csv'), index=False, encoding='utf-8-sig')
valid_long.to_csv(os.path.join(base_dir, 'valid/20per/valid.csv'), index=False, encoding='utf-8-sig')

print("CSV 저장 완료: train/valid 데이터셋 분리 저장")

CSV 저장 완료: train/valid 데이터셋 분리 저장


In [ ]:
# 모델 & 토크나이저 로드
from transformers import MT5ForConditionalGeneration, T5Tokenizer
from peft import LoraConfig, get_peft_model, TaskType

# 베이스 모델명
model_name = "google/mt5-base"
# 토크나이저
tokenizer = T5Tokenizer.from_pretrained(model_name)
# 모델
model = MT5ForConditionalGeneration.from_pretrained(model_name)

# LoRA 설정
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,   # mT5는 Seq2Seq 구조
    r=16,                              # 랭크 (작을수록 가볍고 빠름)
    lora_alpha=32,                     # LoRA scaling factor
    lora_dropout=0.1,                  # 드롭아웃
    target_modules=["q", "v"]          # 주로 attention의 query, value projection에 적용
)

# 기존 베이스 mT5 모델에 LoRA 적용
model = get_peft_model(model, lora_config)

# 적용 확인
model.print_trainable_parameters()

# trainable params: 1,769,472 || all params: 584,170,752 || trainable%: 0.3029
# 전체 파라미터 수: 약 5억 8천만 개 (mT5-base 전체 크기)
# 학습 가능한 파라미터 수: 약 176만 개
# 학습 비율: 약 0.3%만 학습 → 나머지는 고정(frozen)
# 즉, LoRA 덕분에 전체 모델을 학습시키지 않고도 극히 일부 모듈만 학습해서 GPU 메모리와 시간 절약이 가능

trainable params: 1,769,472 || all params: 584,170,752 || trainable%: 0.3029


In [19]:
# # m5T 간단 test

# # 입력(한국어 요약 테스트)
# src_text = "오거돈 전 부산시장의 성추행 사건이 불거진 후 정치권을 중심으로..."
# inputs = tokenizer(src_text, return_tensors='pt', max_length=512, truncation=True)

# # 타겟 요약(학습시 필요) - 정답 요약문을 토큰화, 학습시 labels를 이용해 모델이 출력과 비교하여 loss 계산
# labels = tokenizer("오 전 부산시장의 성추행 사건이 불거졌다.", return_tensors='pt', max_length=128, truncation=True).input_ids

# # Forward pass
# outputs = model(**inputs, labels=labels) # 입력과 정답 요약을 모델에 전달
# loss = outputs.loss # CrossEntropy Loss 값, 모델이 얼마나 잘 요약했는지 측정

# # 요약 생성(추론), 학습된 모델을 이용해 요약문 생성, 첫 단어를 예측할때 상위 4개 후보를 선택
# summary_ids = model.generate(inputs['input_ids'], max_length=128, num_beams=4)
# # 텍스트로 변환, <pad>/<eos> 같은 특수 토큰 제거
# print(tokenizer.decode(summary_ids[0], skip_special_tokens=True))

In [20]:
# CSV 데이터셋 로드
from datasets import load_dataset, concatenate_datasets

# 기본 경로
base_dir = './llm_data/ai_hub_summary_news_r'
# 전체 데이터셋 로드(short/long 모두 포함)
dataset = load_dataset(
    'csv',
    data_files={
        'train_short': os.path.join(base_dir, 'train/2-3sent/train.csv'),
        'train_long': os.path.join(base_dir, 'train/20per/train.csv'),
        'valid_short': os.path.join(base_dir, 'valid/2-3sent/valid.csv'),
        'valid_long': os.path.join(base_dir, 'valid/20per/valid.csv')
    }
)

# 두 split을 합쳐서 하나의 데이터셋으로 만들기
train_concat = concatenate_datasets([
    dataset['train_short'].select(range(100)),
    dataset['train_long'].select(range(100))
])

valid_concat = concatenate_datasets([
    dataset['valid_short'].select(range(100)),
    dataset['valid_long'].select(range(100))
])

print('train_concat : ', len(train_concat))
print('valid_concat : ', len(valid_concat))

train_concat :  200
valid_concat :  200


In [26]:
# 데이터셋 전처리 함수 & 데이터셋 토큰화

# 데이터셋 전처리
def preprocess(batch):
    # summary_type을 prefix로 활용
    inputs = [ f'[{stype.upper()}] {src}' for src, stype in zip(batch['src'], batch['summary_type'])]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    labels = tokenizer(batch['tgt'], max_length=128, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# 합쳐진 데이터셋에 바로 토큰화 적용
tokenized_train = train_concat.map(preprocess, batched=True)
tokenized_valid = valid_concat.map(preprocess, batched=True)

Map: 100%|██████████| 200/200 [00:00<00:00, 258.62 examples/s]


In [29]:
# Trainer 파이프라인
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq

# DataCollator 적용
# Seq2SeqTrainer에서는 DataCollatorForSeq2Seq를 사용하면 자동으로 길이에 대해서 패딩을 맞춰준다
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# 학습 설정
training_args = Seq2SeqTrainingArguments(
    output_dir="./llm_models/results_lora_ai_hub_news_r",       # 결과 저장 경로
    eval_strategy="epoch",                                      # 매 epoch마다 평가
    learning_rate=5e-4,                                         # 학습률, # LoRA는 보통 조금 더 큰 lr 사용 가능
    per_device_train_batch_size=16,                              # 학습 배치 크기
    per_device_eval_batch_size=16,                               # 검증 배치 크기
    num_train_epochs=1,                                         # 학습 epoch 수
    weight_decay=0.01,                                          # 가중치 감쇠
    save_total_limit=2,                                         # 체크포인트 최대 개수
    logging_dir="./llm_models/results_lora_logs_ai_hub_news_r", # 로그 저장 경로
    logging_steps=50,                                           # 로그 출력 주기
    predict_with_generate=True                                  # 평가 시 generate() 사용
)

# Trainer 정의
trainer = Seq2SeqTrainer(
    model=model,    # LoRA 적용 모델
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    processing_class=tokenizer,
    data_collator=data_collator     # HuggingFace Trainer는 배치 단위로 묶을 때 모든 시퀀스 길이가 동일하게 맞추기 위함
)

In [30]:
# 학습 진행
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,18.582268


TrainOutput(global_step=13, training_loss=21.143897423377403, metrics={'train_runtime': 3551.3361, 'train_samples_per_second': 0.056, 'train_steps_per_second': 0.004, 'total_flos': 240896389939200.0, 'train_loss': 21.143897423377403, 'epoch': 1.0})

In [36]:
# 추론
from transformers import pipeline

# 학습된 모델과 토크나이저 로드
summarizer = pipeline( # Hugging Face pipeline은 모델과 토크나이저를 묶어서 간단히 추론 API
    "text2text-generation",
    model=model,
    tokenizer=tokenizer
)

# 테스트 입력 문장
test_text = """
지난주 열린 국제 AI 컨퍼런스에서는 생성형 AI의 윤리적 활용과
기업 내 도입 전략에 대한 다양한 논의가 이루어졌다.
특히 데이터 프라이버시와 저작권 문제에 대한 해결책이 주요 의제로 다뤄졌다.
"""

# 요약 생성
# summary = summarizer( # Greedy Decoding (탐욕적 디코딩)
#     f"[SHORT] {test_text}", # summary_type prefix 활용, prefix([SHORT], [LONG])
#     max_length=128,
#     min_length=20,
#     do_sample=False         # 샘플링 없이 greedy decoding으로 결정적 결과
# )
# Beam Search (빔 탐색)
summary = summarizer(
    f"[SHORT] {test_text}",
    max_length=128,
    num_beams=5,      # 빔 개수
    early_stopping=True
)
# Sampling?(샘플링)
# summary = summarizer(
#     f"[SHORT] {test_text}",
#     max_length=128,
#     do_sample=True,   # 샘플링 켬
#     top_k=50,         # 확률 상위 50개 중 선택
#     top_p=0.95,       # 누적 확률 95% 내에서 선택
#     num_return_sequences=3
# )

# summary[0] 리스트의 첫 번째 결과 (딕셔너리), summary[0]["generated_text"] 딕셔너리 안에서 모델이 실제로 생성한 요약문 텍스트
print('요약 결과:', summary[0]["generated_text"])
# for i, s in enumerate(summary_2):
#     print(f"요약 {i+1}: {s['generated_text']}")

Device set to use cpu


Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


요약 결과: <extra_id_0> AI의 윤리적 활용


In [37]:
# LoRA 적용된 모델 저장
model.save_pretrained("./llm_models/summary_model_ai_hub_news_r_lora")
tokenizer.save_pretrained("./llm_models/summary_model_ai_hub_news_r_lora")

('./llm_models/summary_model_ai_hub_news_r_lora/tokenizer_config.json',
 './llm_models/summary_model_ai_hub_news_r_lora/special_tokens_map.json',
 './llm_models/summary_model_ai_hub_news_r_lora/spiece.model',
 './llm_models/summary_model_ai_hub_news_r_lora/added_tokens.json')

In [38]:
from transformers import MT5ForConditionalGeneration, T5Tokenizer

# 모델 로드
model = MT5ForConditionalGeneration.from_pretrained(
    "./llm_models/summary_model_ai_hub_news_r_lora"
)

# 토크나이저 로드
tokenizer = T5Tokenizer.from_pretrained(
    "./llm_models/summary_model_ai_hub_news_r_lora"
)

In [39]:
# 모델 로드 & 추론
from transformers import pipeline

# 학습된 모델과 토크나이저 로드
summarizer = pipeline( # Hugging Face pipeline은 모델과 토크나이저를 묶어서 간단히 추론 API
    "text2text-generation",
    model=model,
    tokenizer=tokenizer
)

# 테스트 입력 문장
test_text = """
지난주 열린 국제 AI 컨퍼런스에서는 생성형 AI의 윤리적 활용과
기업 내 도입 전략에 대한 다양한 논의가 이루어졌다.
특히 데이터 프라이버시와 저작권 문제에 대한 해결책이 주요 의제로 다뤄졌다.
"""

# 요약 생성
# summary = summarizer( # Greedy Decoding (탐욕적 디코딩)
#     f"[SHORT] {test_text}", # summary_type prefix 활용, prefix([SHORT], [LONG])
#     max_length=128,
#     min_length=20,
#     do_sample=False         # 샘플링 없이 greedy decoding으로 결정적 결과
# )
# Beam Search (빔 탐색)
summary = summarizer(
    f"[SHORT] {test_text}",
    max_length=128,
    num_beams=5,      # 빔 개수
    early_stopping=True
)
# Sampling?(샘플링)
# summary = summarizer(
#     f"[SHORT] {test_text}",
#     max_length=128,
#     do_sample=True,   # 샘플링 켬
#     top_k=50,         # 확률 상위 50개 중 선택
#     top_p=0.95,       # 누적 확률 95% 내에서 선택
#     num_return_sequences=3
# )

# summary[0] 리스트의 첫 번째 결과 (딕셔너리), summary[0]["generated_text"] 딕셔너리 안에서 모델이 실제로 생성한 요약문 텍스트
print('요약 결과:', summary[0]["generated_text"])
# for i, s in enumerate(summary_2):
#     print(f"요약 {i+1}: {s['generated_text']}")

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


요약 결과: <extra_id_0> AI의 윤리적 활용
